In [ ]:
!pip install torch torchvision torchnet xmltodict pycocotools

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 30.8 MB/s eta 0:00:00
  Created wheel for torchnet: filename=torch

In [ ]:
import os
import torch
import torch.nn as nn
import torchvision
from torchvision.models.detection import retinanet_resnet50_fpn
from torchvision.models.detection.retinanet import RetinaNetHead
import xmltodict
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
data_paths = {
    'Missing_hole': '/content/drive/My Drive/PCB/PCB_DATASET/images/Missing_hole',
    'Mouse_bite': '/content/drive/My Drive/PCB/PCB_DATASET/images/Mouse_bite',
    'Open_circuit': '/content/drive/My Drive/PCB/PCB_DATASET/images/Open_circuit',
    'Short': '/content/drive/My Drive/PCB/PCB_DATASET/images/Short',
    'Spurious_copper': '/content/drive/My Drive/PCB/PCB_DATASET/images/Spurious_copper',
    'Spur': '/content/drive/My Drive/PCB/PCB_DATASET/images/Spur'
}

annot_paths = {
    'Missing_hole': '/content/drive/My Drive/PCB/PCB_DATASET/Annotations/Missing_hole',
    'Mouse_bite': '/content/drive/My Drive/PCB/PCB_DATASET/Annotations/Mouse_bite',
    'Open_circuit': '/content/drive/My Drive/PCB/PCB_DATASET/Annotations/Open_circuit',
    'Short': '/content/drive/My Drive/PCB/PCB_DATASET/Annotations/Short',
    'Spurious_copper': '/content/drive/My Drive/PCB/PCB_DATASET/Annotations/Spurious_copper',
    'Spur': '/content/drive/My Drive/PCB/PCB_DATASET/Annotations/Spur'
}
class_names = ['Missing_hole', 'Mouse_bite', 'Open_circuit', 'Short', 'Spurious_copper', 'Spur']
num_classes = len(class_names) + 1  # +1 for background

Mounted at /content/drive


In [ ]:
# Custom Dataset
class PCBDataset(Dataset):
    def __init__(self, data_paths, annot_paths, transform=None):
        self.data_paths = data_paths
        self.annot_paths = annot_paths
        self.transform = transform
        self.images = []
        self.targets = []

        for cls in data_paths.keys():
            img_dir = data_paths[cls]
            annot_dir = annot_paths[cls]
            for img_file in os.listdir(img_dir):
                if img_file.endswith(('.jpg', '.png')):
                    img_path = os.path.join(img_dir, img_file)
                    annot_file = os.path.join(annot_dir, img_file.replace('.jpg', '.xml').replace('.png', '.xml'))
                    if os.path.exists(annot_file):
                        self.images.append(img_path)
                        self.targets.append(self.parse_voc_xml(annot_file, cls))

    def parse_voc_xml(self, xml_file, cls_name):
        with open(xml_file) as f:
            xml = xmltodict.parse(f.read())
        objects = xml['annotation']['object']
        if not isinstance(objects, list):
            objects = [objects]
        boxes = []
        labels = []
        for obj in objects:
            bbox = obj['bndbox']
            boxes.append([float(bbox['xmin']), float(bbox['ymin']),
                         float(bbox['xmax']), float(bbox['ymax'])])
            labels.append(class_names.index(cls_name) + 1)  # +1 because 0 is background
        return {
            'boxes': torch.tensor(boxes, dtype=torch.float32),
            'labels': torch.tensor(labels, dtype=torch.int64)
        }

    def __len__(self):
        return len(self.images)
    def __getitem__(self, idx):
        img = torchvision.io.read_image(self.images[idx]).float() / 255.0
        target = self.targets[idx]
        if self.transform:
            img = self.transform(img)
        return img, target

In [ ]:
# Data Loading
transform = torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
dataset = PCBDataset(data_paths, annot_paths, transform=transform)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, num_workers=2,collate_fn=lambda x: tuple(zip(*x)))
val_loader = DataLoader(val_dataset, batch_size=2, shuffle=False, num_workers=2,collate_fn=lambda x: tuple(zip(*x)))

In [ ]:
# Model Setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = retinanet_resnet50_fpn(pretrained=True).to(device)
# Replace the classification head for our number of classes (6 + 1 background = 7)
model.head = RetinaNetHead(in_channels=256,num_anchors=model.head.classification_head.num_anchors,num_classes=num_classes).to(device)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=RetinaNet_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=RetinaNet_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_coco-eeacb38b.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_coco-eeacb38b.pth
100%|██████████| 130M/130M [00:00<00:00, 214MB/s]


In [ ]:
def evaluate(model, data_loader, device):
    model.eval()
    all_preds = []
    all_targets = []
    with torch.no_grad():
        for images, targets in data_loader:
            images = list(img.to(device) for img in images)
            outputs = model(images)
            for i, output in enumerate(outputs):
                pred_boxes = output['boxes'].cpu().numpy()
                pred_labels = output['labels'].cpu().numpy()
                pred_scores = output['scores'].cpu().numpy()
                # Filter predictions with score > 0.5 (or lower threshold for early epochs)
                threshold = 0.3 if epoch < 2 else 0.5  # Lower threshold for first 2 epochs
                mask = pred_scores > threshold
                pred_labels = pred_labels[mask] if mask.any() else []
                all_preds.extend(pred_labels)
                all_targets.extend(targets[i]['labels'].cpu().numpy())
    # Calculate metrics only if there are predictions
    if len(all_preds) > 0 and len(all_targets) > 0:
        precision, recall, f1, _ = precision_recall_fscore_support(all_targets, all_preds, average='weighted')
        conf_matrix = confusion_matrix(all_targets, all_preds)
        # Simplified mAP calculation
        mAP = np.mean([precision, recall])
    else:
        precision, recall, f1 = 0.0, 0.0, 0.0
        conf_matrix = np.zeros((num_classes, num_classes), dtype=int)
        mAP = 0.0
        print("Warning: No valid predictions found. Metrics set to 0. Model may need more training.")
    return mAP, precision, recall, f1, conf_matrix

In [ ]:
from tqdm import tqdm
num_epochs = 10
for epoch in range(num_epochs):
    # Set model to training mode
    model.train()
    total_loss = 0
    # Create progress bar for images in the epoch
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=True)
    for images, targets in progress_bar:
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        optimizer.zero_grad()
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        losses.backward()
        optimizer.step()
        total_loss += losses.item()
        # Update progress bar with current loss
        avg_loss = total_loss / (progress_bar.n + 1)
        progress_bar.set_postfix({'Box Loss': f"{loss_dict['classification'].item():.3f}",
                                 'Cls Loss': f"{loss_dict['bbox_regression'].item():.3f}",
                                 'Total Loss': f"{avg_loss:.3f}"})
    avg_epoch_loss = total_loss / len(train_loader)
    print(f"\nEpoch {epoch+1}/{num_epochs} - Avg Loss: {avg_epoch_loss:.3f}")
    mAP, precision, recall, f1, conf_matrix = evaluate(model, val_loader, device)
    print(f"Metrics - mAP: {mAP:.3f}, Precision: {precision:.3f}, Recall: {recall:.3f}, F1: {f1:.3f}")
    print(f"Images: {len(val_dataset)}, Instances: {sum(len(t['labels']) for _, t in val_dataset)}")
    print("-" * 80)
mAP, precision, recall, f1, conf_matrix = evaluate(model, val_loader, device)
print(f"\nFinal Results after {num_epochs} epochs - mAP: {mAP:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")
print(f"Images: {len(val_dataset)}, Instances: {sum(len(t['labels']) for _, t in val_dataset)}")
plt.figure(figsize=(10,8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

Epoch 1/10:  45%|████▍     | 124/277 [1:18:58<1:33:00, 36.47s/it, Box Loss=1.171, Cls Loss=0.676, Total Loss=1.868]

In [ ]:
mAP, precision, recall, f1, conf_matrix = evaluate(model, val_loader, device)
print(f"Final Results - mAP: {mAP:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")